In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.amp import GradScaler, autocast # NEW: Use robust torch.amp
from torch.utils.data import Dataset, DataLoader

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import glob
from collections import defaultdict
from PIL import Image

import math
import shutil
import random
from typing import Callable, Optional, Dict, Any, Tuple, List

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler


/home/mohammed/miniconda3/envs/bioenv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/mohammed/miniconda3/envs/bioenv/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/mohammed/miniconda3/envs/bioenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [72]:
from utils.config import CFG
def get_id(path):
    return os.path.splitext(os.path.basename(path))[0]

base_directory = CFG.DATA_DIR
train_csv = pd.read_csv(
    os.path.join(base_directory , "train.csv")
)
train_imgs_path = glob.glob(\
    os.path.join(base_directory , "train" , "*.jpg")
)

dict_ = defaultdict(list)

for path in train_imgs_path:
    id_ = get_id(path)
    cols_to_include = ["target_name" , "target" , "Sampling_Date" , "State" ,"Pre_GSHH_NDVI" ]
    img_info = train_csv[train_csv["sample_id"].str.split("_" , expand=True)[0]==id_][cols_to_include].copy()
    extra_info = img_info[[ "Sampling_Date" , "State" ,"Pre_GSHH_NDVI"]].iloc[0,:].to_dict()
    info_dict = img_info.set_index("target_name").to_dict()['target']
    info_dict["id"] = id_
    info_dict["image_path"] = path
    info_dict.update(extra_info)
    for k  , v in info_dict.items():
        dict_[k].append(v)

df = pd.DataFrame(dict_)

In [ ]:
from utils.Kfold import create_folds

s = create_folds(
    df , strategy="stratified",random_state=42, stratify_col="State" , n_splits=5
)
df["fold"] = s



Index([  7,  11,  13,  14,  16,  17,  33,  36,  48,  50,  55,  58,  76,  77,
        78,  79,  80,  81,  85,  86,  94,  95,  96,  97, 105, 114, 122, 123,
       129, 130, 131, 140, 150, 166, 170, 174, 178, 181, 189, 190, 201, 203,
       211, 212, 217, 218, 219, 228, 233, 235, 237, 240, 248, 254, 264, 286,
       302, 303, 315, 319, 320, 323, 326, 330, 343, 344, 345, 347, 349, 350,
       353, 354],
      dtype='int64')